In [1]:
#各ライブラリのimport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
#重回帰分析(trainのみ)
#pwrカラム追加版
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)
#pwrカラム作成
df_train['pwr'] = df_train['weight']/df_train['horsepower']
#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正

#エンコーディング
df_number = pd.get_dummies(df_train , columns = ['origin' , 'manufacturers name'] , dtype=int)

#標準化
#目的変数の標準化は必要ない
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_number.drop(['id' , 'car name', 'mpg','cylinders' , 'weight','horsepower'] , axis = 1)
df_train_std = pd.DataFrame(scaler.fit_transform(df_train_drop), columns = df_train_drop.columns)

#特徴量の分割
features = df_train_std
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
target = df_train['mpg']
print(target)
#print(df_train_std.describe())
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.2 , random_state = 0)
#ライブラリのimport
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#学習と評価
model.fit(features_train , target_train)
pred = model.predict(features_test).reshape(-1,1)

#評価
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(pred,target_test)) 

0      29.0
1      31.9
2      19.0
3      28.0
4      37.7
       ... 
194    40.8
195    20.2
196    16.0
197    43.4
198    26.0
Name: mpg, Length: 199, dtype: float64


471535892278.3837

In [17]:
#get_dummiesでエンコードver
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
df_train_test.isnull().sum()

df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出
#欠損値補完
#amc concord dl
df_train_test.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean()
#renault 18
df_train_test.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean()
#renault lecar deluxe
df_train_test.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean()
#ford pinto
df_train_test.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean()
#ford maverick
df_train_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean()
#ford mustang cobra
df_train_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean()
#pwrカラム作成
df_train_test['pwr'] = df_train_test['weight']/df_train_test['horsepower']
#メーカーカラム作成
df_split = df_train_test['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({"toyouta":"toyota", "vw":"volkswagen", "vokswagen":"volkswagen",  "chevroelt":"chevrolet" ,  "chevy":"chevrolet","mercury":"ford", "datsun":"nissan", "maxda":"mazda",  "mercedes":"mercedes-benz"})#カラム名の修正

#one-hot
#get_dummies(扱うdf,columns=カテゴリ変数を含むカラム,dtype=データ型)
#元のdfに数値化したカラムを結合してくれるから便利
df_number = pd.get_dummies(df_train_test , columns = ['origin' , 'manufacturers name'] , dtype=int)
#:199だと198番目までが選択される
df_train = df_number.iloc[:199 , :]
df_test = df_number.iloc[199: , :]

#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop(['id' , 'car name'  , 'mpg','weight' ,'horsepower','cylinders'] , axis = 1)
df_test_drop = df_test.drop(['id' , 'car name' , 'mpg','weight' ,'horsepower','cylinders'] , axis = 1)

scaler.fit(df_train_drop)
df_train_std= pd.DataFrame(scaler.transform(df_train_drop), columns = df_train_drop.columns)
df_test_std = pd.DataFrame(scaler.transform(df_test_drop), columns = df_test_drop.columns)
print(df_train_std)

#目的変数の分離
features_train = df_train_std
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
target_train = df_train['mpg']
print(df_train['mpg'])
features_test = df_test_std

#学習
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(features_train , target_train)
#予測
df_pred = pd.DataFrame(model.predict(features_test))
print(df_pred)
#print(df_pred[df_pred <= 0])
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')

df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)

#評価
#multiple regression analysisの頭文字をとってMRA_sample_submit.csv
df_pred.to_csv('MRA_sample_submit.csv' , header = False , index = False)

     displacement  model year       pwr  origin_1  origin_2  origin_3  \
0       -0.492207    1.537995  0.119333  0.794448 -0.485824 -0.493710   
1       -0.960864    0.747140 -0.494432 -1.258736  2.058359 -0.493710   
2       -0.278255   -0.043716 -0.490901 -1.258736 -0.485824  2.025479   
3       -0.950675   -0.570953 -0.240195  0.794448 -0.485824 -0.493710   
4       -0.960864    1.274377  0.745209 -1.258736 -0.485824  2.025479   
..            ...         ...       ...       ...       ...       ...   
194     -1.001616    1.010758  0.619621 -1.258736 -0.485824  2.025479   
195      1.209220    0.483521 -0.792109  0.794448 -0.485824 -0.493710   
196      1.229596   -1.625426 -1.374619  0.794448 -0.485824 -0.493710   
197     -0.950675    1.010758  3.990464 -1.258736  2.058359 -0.493710   
198     -0.869170   -0.834571 -0.899744 -1.258736  2.058359 -0.493710   

     manufacturers name_amc  manufacturers name_audi  manufacturers name_bmw  \
0                 -0.295689                